In [202]:
import numpy as np
import random
from collections import namedtuple
from itertools import combinations
import matplotlib.pyplot as plt

## Common tests

In [203]:
problem = np.load('lab2/problem_r1_50.npy')
num_cities = problem.shape[0]

In [204]:
PROBLEM_SIZE = num_cities
Individual = namedtuple('Individual', ['genotype', 'fitness'])

In [205]:
PROBLEM_SIZE

50

In [206]:
# return the list of distances according to the current solution
def get_distances(current_sol):
    distances = []
    for idx, city_a in enumerate(current_sol):
        city_b_idx = idx + 1
        if city_b_idx == len(current_sol):  # Also fixed this
            city_b_idx = 0

        city_b = current_sol[city_b_idx]
        
        d = problem[city_a, city_b]
        distances.append(d)
    
    return distances

# return the total distance for the current solution
def tot_distance(distances):
    return sum(distances)

# Evaluate a tour and return fitness
def evaluate_tour(tour):
    return tot_distance(get_distances(tour))


In [207]:
def tournament_selection(population, tau=5):
    """Tournament selection - good for minimization"""
    pool = random.sample(population, min(tau, len(population)))
    return min(pool, key=lambda i: i.fitness)

In [208]:
def inversion_mutation(genotype):
    """Reverse a random segment of the tour"""
    new_genotype = genotype[:]
    idx1, idx2 = sorted(random.sample(range(len(genotype)), 2))
    new_genotype[idx1:idx2+1] = reversed(new_genotype[idx1:idx2+1])
    return new_genotype


In [209]:
def order_crossover(p1, p2):
    """Order Crossover (OX1) - preserves relative order"""
    size = len(p1)
    start, end = sorted(random.sample(range(size), 2))
    
    child = [None] * size
    child[start:end] = p1[start:end]
    
    p2_cities = [city for city in p2 if city not in child]
    pos = 0
    for i in range(size):
        if child[i] is None:
            child[i] = p2_cities[pos]
            pos += 1
    
    return child

In [210]:
def two_opt_local_search(tour, max_iterations=50):
    """2-opt local search for tour improvement"""
    best_tour = tour[:]
    best_distance = evaluate_tour(best_tour)
    improved = True
    iterations = 0
    
    while improved and iterations < max_iterations:
        improved = False
        iterations += 1
        
        for i in range(1, len(tour) - 1):
            for j in range(i + 1, len(tour)):
                # Create new tour by reversing segment
                new_tour = best_tour[:i] + best_tour[i:j+1][::-1] + best_tour[j+1:]
                new_distance = evaluate_tour(new_tour)
                
                if new_distance < best_distance:
                    best_tour = new_tour
                    best_distance = new_distance
                    improved = True
                    break
            if improved:
                break
    
    return best_tour

In [211]:
def random_initialization(size, n):
    """Random permutations"""
    population = []
    for _ in range(n):
        genotype = list(range(size))
        random.shuffle(genotype)
        fitness = evaluate_tour(genotype)
        population.append(Individual(genotype, fitness))
    return population

def nearest_neighbor_initialization(size, n):
    """Greedy nearest neighbor heuristic"""
    population = []
    
    for _ in range(n):
        start = random.randint(0, size - 1)
        tour = [start]
        unvisited = set(range(size)) - {start}
        
        while unvisited:
            current = tour[-1]
            # Find nearest unvisited city
            nearest = min(unvisited, key=lambda c: problem[current, c])
            tour.append(nearest)
            unvisited.remove(nearest)
        
        fitness = evaluate_tour(tour)
        population.append(Individual(tour, fitness))
    
    return population

def mixed_initialization(size, n):
    """Mix of random and greedy initialization"""
    population = []
    # 30% greedy, 70% random
    greedy_count = int(n * 0.3)
    population.extend(nearest_neighbor_initialization(size, greedy_count))
    population.extend(random_initialization(size, n - greedy_count))
    return population

In [212]:
def genetic_algorithm_with_local_search(config):
    """Genetic Algorithm with Local Search"""
    print(f"\n{'='*60}")
    print(f"Genetic Algorithm with Local Search")
    print(f"{'='*60}")
    
    # Initialize
    population = config['init_func'](num_cities, config['pop_size'])
    
    # Apply local search to initial population
    print("Applying local search to initial population...")
    improved_pop = []
    for ind in population:
        improved_genotype = two_opt_local_search(ind.genotype, max_iterations=20)
        improved_fitness = evaluate_tour(improved_genotype)
        improved_pop.append(Individual(improved_genotype, improved_fitness))
    population = improved_pop
    
    best_ever = min(population, key=lambda i: i.fitness)
    history = [best_ever.fitness]
    
    for gen in range(config['max_gen']):
        offspring = []
        
        # Generate offspring
        for _ in range(config['offspring_size']):
            if random.random() < config['mutation_rate']:
                parent = config['selection'](population)
                child_genotype = config['mutation'](parent.genotype)
            else:
                p1 = config['selection'](population)
                p2 = config['selection'](population)
                child_genotype = config['crossover'](p1.genotype, p2.genotype)
            
            # Apply local search to some offspring
            if random.random() < config['local_search_rate']:
                child_genotype = two_opt_local_search(child_genotype, max_iterations=10)
            
            fitness = evaluate_tour(child_genotype)
            offspring.append(Individual(child_genotype, fitness))
        
        # Elitism + survival selection
        population.extend(offspring)
        population = sorted(population, key=lambda i: i.fitness)[:config['pop_size']]
        
        current_best = population[0]
        if current_best.fitness < best_ever.fitness:
            best_ever = current_best
        
        history.append(best_ever.fitness)
        
        if (gen + 1) % 50 == 0:
            print(f"Gen {gen+1}: Best = {best_ever.fitness:.2f}")
    
    print(f"\nFinal Best: {best_ever.fitness:.2f}")
    return best_ever, history

In [213]:
def run_experiments():
    max_gen = 200
    if PROBLEM_SIZE > 50:
        max_gen = 100
    # Base configuration
    base_config = {
        'pop_size': 100,
        'offspring_size': 50,
        'max_gen': max_gen,
        'mutation_rate': 0.5,
        'mutation': inversion_mutation,
        'crossover': order_crossover,
        'selection': tournament_selection,
        'init_func': mixed_initialization,
        'local_search_rate': 0.2
    }
    
    genetic_algorithm_with_local_search(base_config)
    

In [214]:
run_experiments()


Genetic Algorithm with Local Search
Applying local search to initial population...
Gen 50: Best = 557.54
Gen 100: Best = 555.06
Gen 150: Best = 555.06
Gen 200: Best = 555.06

Final Best: 555.06
